<a href="https://colab.research.google.com/github/kmenesesrivera/codonusagebias/blob/main/Codon_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
import joblib
from google.colab import files
from sklearn.ensemble import RandomForestClassifier 
import pickle


data1 = pd.read_csv('https://drive.google.com/uc?export=download&id=1Z4v43cvTwp920NyOdboDKP7_ytC_0tBC')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data1.head(5)

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [3]:
data1.dtypes

Kingdom         object
DNAtype          int64
SpeciesID        int64
Ncodons          int64
SpeciesName     object
                ...   
GAA            float64
GAG            float64
UAA            float64
UAG            float64
UGA            float64
Length: 69, dtype: object

In [4]:
# Verificamos que aún hay columnas que deberían ser númericas.

cols = data1.columns[data1.dtypes.eq(object)] 
cols

Index(['Kingdom', 'SpeciesName', 'UUU', 'UUC'], dtype='object')

In [5]:
data1.shape

(13028, 69)

In [6]:
data1.isna().sum().sort_values()

Kingdom    0
UCA        0
UCG        0
AGU        0
AGC        0
          ..
GUG        0
GUA        0
GUC        0
GCG        0
UGA        0
Length: 69, dtype: int64

In [7]:
# Se observa que hay datos str que no permiten manipular los demás como numéricos.
data1[['UUU', 'UUC']] = data1[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
null_UUU = data1['UUU'].isna().sum()
null_UUC= data1['UUC'].isna().sum()
print ("Cantidad de datos nulos en codon UUU ",null_UUU)
print ("Cantidad de datos nuls en codon UUC ",null_UUC)


Cantidad de datos nulos en codon UUU  2
Cantidad de datos nuls en codon UUC  1


In [8]:
#Lectura del dataset
 
data1.describe

data1.head(5)
 

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [9]:
def preprocess_dataset(dataset, save_metadata=True):
  
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()
  
  # Está columna contiene datos descriptivos, por tanto será transformada a str.

  preprocessed_dataset['SpeciesName'] = preprocessed_dataset['SpeciesName'].astype(str)
  preprocessed_dataset['SpeciesName']
 
  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)
 
  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)
  
  #Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
    median_of_numeric_columns = preprocessed_dataset[numeric_columns].median()
    preprocessed_dataset[numeric_columns] = preprocessed_dataset[numeric_columns].fillna(median_of_numeric_columns)

  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

  #Determinamos aquellas variables que tengan un porcentaje de registros nulos por cada variable mayor al valor de 0.7
  threshold = 0.7

  null_percentages = preprocessed_dataset[categorical_columns].isna().sum() / len(preprocessed_dataset)

  criteria = null_percentages > threshold

  columns_to_filter = null_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  if save_metadata:
    #Guardamos información que consideremos imporante. **El formato pickle nos permite guardar objetos de python***
    with open("numeric_columns.pkl", "wb") as f:
      pickle.dump(numeric_columns, f)
    
    with open("categorical_columns.pkl", "wb") as f:
      pickle.dump(categorical_columns, f)
    
    with open("dictionary_of_columns_with_index_to_categorical.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_index_to_categorical, f)
    
    with open("dictionary_of_columns_with_categorical_to_index.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_categorical_to_index, f)
  
  return preprocessed_dataset

In [10]:
#Transformamos los datos a valores numéricos
preprocessed_dataset = preprocess_dataset(data1)

#print ("preprocessed_dataset", preprocessed_dataset.describe) 

#Se redujo la cantidad de genomas luego de eliminar duplicados a 12991, se reemplaza nulos por la mediana, moda o se puede eliminar registros.

#print (preprocessed_dataset.isna().sum().sort_values())

In [11]:
preprocessed_dataset.describe

<bound method NDFrame.describe of        Kingdom  DNAtype   Ncodons      UUU  ...      GAG      UAA      UAG      UGA
0            9        0      1995  0.01654  ...  0.04361  0.00251  0.00050  0.00000
1            9        0      1474  0.02714  ...  0.04410  0.00271  0.00068  0.00000
2            9        0      4862  0.01974  ...  0.02468  0.00391  0.00000  0.00144
3            9        0      1915  0.01775  ...  0.03446  0.00261  0.00157  0.00000
4            9        0     22831  0.02816  ...  0.03679  0.00000  0.00044  0.00131
...        ...      ...       ...      ...  ...      ...      ...      ...      ...
13023        7        0      1097  0.02552  ...  0.04102  0.00091  0.00091  0.00638
13024        7        1      2067  0.01258  ...  0.00677  0.00242  0.00097  0.01887
13025        7        1      1686  0.01423  ...  0.00297  0.00356  0.00119  0.02017
13026        7        0  40662582  0.01757  ...  0.03959  0.00099  0.00079  0.00156
13027        7        1   8998998  0.01778

In [12]:
x = preprocessed_dataset.drop("Kingdom", axis=1)
x = preprocessed_dataset.drop("DNAtype", axis=1)
y = preprocessed_dataset["Kingdom"]  
y2 = preprocessed_dataset["DNAtype"] 


In [13]:
 #Selección de Características

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

filtrado = SelectKBest(mutual_info_classif, k=3).fit(x, y)

In [14]:
filtrado.scores_

array([1.9006329 , 0.27856014, 0.21517164, 0.23834153, 0.24657704,
       0.2721422 , 0.22766302, 0.27351454, 0.48384315, 0.23192416,
       0.21100844, 0.28477362, 0.30257539, 0.25069125, 0.22502464,
       0.15187516, 0.1533248 , 0.24499133, 0.18658977, 0.28811815,
       0.15368324, 0.32420818, 0.18137754, 0.22920197, 0.31514394,
       0.28832239, 0.3050538 , 0.25485281, 0.27969618, 0.20606499,
       0.12362841, 0.23914512, 0.19985247, 0.24282911, 0.25464307,
       0.26213345, 0.17519721, 0.19052721, 0.25177895, 0.33275427,
       0.25931914, 0.20321743, 0.12514707, 0.20466868, 0.32233914,
       0.23472852, 0.16781138, 0.33577438, 0.2357943 , 0.20898512,
       0.1940669 , 0.30276959, 0.39082026, 0.29886386, 0.29804887,
       0.23339844, 0.24221105, 0.4441011 , 0.46378954, 0.35078053,
       0.23725073, 0.27141378, 0.3242554 , 0.20427753, 0.08303098,
       0.45725223])

In [15]:
filtrado.pvalues_

In [16]:
x_new= filtrado.transform(x)
x_new[:6]

array([[9.000e+00, 1.000e-03, 3.559e-02],
       [9.000e+00, 2.040e-03, 3.596e-02],
       [9.000e+00, 1.028e-02, 2.489e-02],
       [9.000e+00, 9.400e-03, 1.671e-02],
       [9.000e+00, 4.730e-03, 1.734e-02],
       [9.000e+00, 1.782e-02, 1.710e-03]])

In [17]:
filtrado2 = SelectKBest(mutual_info_classif, k=60).fit(x, y2)

In [18]:
filtrado2.scores_

array([0.40388937, 0.18626646, 0.18301357, 0.15823009, 0.24389902,
       0.23862555, 0.19056793, 0.20538403, 0.46267312, 0.17193469,
       0.24143265, 0.19273589, 0.31504524, 0.27574462, 0.10129945,
       0.16331816, 0.20917967, 0.31912314, 0.09670497, 0.17704709,
       0.13526211, 0.28091523, 0.07457589, 0.14218957, 0.18152661,
       0.26172454, 0.33594043, 0.19615478, 0.19976235, 0.12404275,
       0.09580807, 0.14647049, 0.10521396, 0.20529645, 0.23080991,
       0.214313  , 0.21586648, 0.0890177 , 0.17759283, 0.24956996,
       0.27355461, 0.15508772, 0.10984685, 0.1279072 , 0.43804812,
       0.14231695, 0.1564824 , 0.16958558, 0.21198318, 0.15760029,
       0.15601517, 0.2143281 , 0.49021633, 0.26956282, 0.22301816,
       0.23852405, 0.18907567, 0.29706481, 0.35633921, 0.30450247,
       0.31278839, 0.23028881, 0.46117725, 0.16253138, 0.06187115,
       0.51239787])

In [19]:
filtrado2.pvalues_

In [20]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)


In [21]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10392, 66)
Test shape: (2599, 66)


In [22]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(5,6)
}

xgb_Model = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid = GridSearchCV (estimator= xgb_Model, param_grid = param_grid )
 
xgb_Grid.fit(x_train,y_train)
 
print ("optimal max_depth",xgb_Grid.best_estimator_.max_depth) 
print ("optimal xgb_Grid.best_score_",xgb_Grid.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y_train)


optimal max_depth 5
optimal xgb_Grid.best_score_ 0.9992301677575981


In [23]:
xgb_Grid.score(x_train, y_train)

0.9998075442648191

In [24]:
xgb_Grid.score(x_test, y_test)

0.9984609465178915

In [25]:

#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, test_size=0.2, random_state=70)


In [26]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10392, 66)
Test shape: (2599, 66)


In [27]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(9,10)
}

xgb_Model2 = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid2 = GridSearchCV (estimator= xgb_Model2, param_grid = param_grid )
 
xgb_Grid2.fit(x_train,y2_train)
 
print ("optimal max_depth",xgb_Grid2.best_estimator_.max_depth)
print ("optimal xgb_Grid2.best_score_",xgb_Grid2.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y2_train)
xgb_Grid2.score(x_test, y2_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


optimal max_depth 9
optimal xgb_Grid2.best_score_ 0.9867209609269281


0.9899961523662947

In [28]:
xgb_Grid2.score(x_train, y2_train)

0.9927829099307159

In [32]:
import seaborn as sns
from sklearn.metrics import roc_auc_score
X=[]
Y=[]
train_auc= [] 
rf_Model= XGBClassifier(learning_rate=0.01,max_depth=9 ,n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
rf_Model.fit(x_train,y_train)
pred_train= rf_Model.predict(x_train)
X.append(80)
Y.append(9)
train_auc.append(roc_auc_score(y_train,pred_train)) 

ValueError: ignored

In [31]:

y_pred_proba = rfcl.predict_proba(x_test)[::,1]
fpr, tpr, th = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.title("Curva ROC y AUC")
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend(loc=4)
plt.show()

NameError: ignored

In [58]:
# LightGBM
import lightgbm as lgb

from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV


# Set params
# Scores ~0.784 (without tuning and early stopping)
params = {'boosting_type': 'gbdt',
          'max_depth' : 10,
          'objective': 'binary',
          'nthread': -1, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 2,
          'min_child_samples': 6,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# Create parameters to search
gridParams = {
    'learning_rate': [0.1],
    'n_estimators': [91],
    'num_leaves': [19],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [30], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.9],
    'reg_alpha' : [1],
    'reg_lambda' : [1.2],
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)
# Run the grid
grid.fit(x_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)


{'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'n_estimators': 91, 'num_leaves': 19, 'objective': 'binary', 'random_state': 30, 'reg_alpha': 1, 'reg_lambda': 1.2, 'subsample': 0.9}
0.9944187836797537


In [59]:
print(grid.score(x_train, y_train))
print(grid.score(x_test, y_test))

0.9993264049268669
0.9957676029242016
